In [14]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 4.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp38-cp38-macosx_10_9_x86_64.whl (35 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached multiprocess-0.70.14-py38-none-any.whl (132 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached fsspec-2023.1.0-py3-none-any.whl (143 kB)
  Using cached pyarrow-11.0.0-cp38-cp38-macosx_10_14_x86_64.whl (24.4 MB)
  Using cached yarl-1.8.2-cp38-cp38-macosx_10_9_x86_64.whl (60 kB)
  Using cached multidict-6.0.4-cp38-cp38-macosx_10_9_x86_64.whl (29 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-macosx_10_9_x86_64.whl (36 kB)
  Attempting uninstall: fssp

In [7]:
# huggingface datatsets
from datasets import load_dataset

In [41]:
silicone_data_valid = load_dataset('silicone', 'oasis', split = 'validation').to_pandas()

In [42]:
silicone_data_train = load_dataset('silicone', 'oasis', split = 'train').to_pandas()
silicone_data_test = load_dataset('silicone', 'oasis', split = 'test').to_pandas()

In [43]:
# The one hot encoding of each 42 labels
from sklearn.preprocessing import LabelBinarizer
ohencoder=LabelBinarizer()
# Training data
trainlabels=silicone_data_train.copy()
trainlabels=trainlabels.pop('Dialogue_Act')
trainlabels=trainlabels.values

validlabels=silicone_data_valid.copy()
validlabels=validlabels.pop('Dialogue_Act')
validlabels=validlabels.values


In [44]:
#Train data
trainlabels=ohencoder.fit_transform(trainlabels)

# Test data
testlabels=ohencoder.fit_transform(silicone_data_test['Dialogue_Act'].values)

# Validation data
validationlabels=ohencoder.transform(validlabels)

In [45]:
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import numpy as np

In [46]:
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 1

In [47]:
X_train = silicone_data_train
X_test = silicone_data_test

In [48]:
trainfeatures= silicone_data_train.copy()
trainfeatures.drop(columns=['Speaker','Label','Dialogue_Act','Idx'],inplace=True)

trainfeatures=trainfeatures.values
#trainlabels=silicone_data_train.copy()
#trainlabels=trainlabels.pop('Dialogue_Act')
#trainlabels=trainlabels.values

#validlabels=silicone_data_valid.copy()
#validlabels=validlabels.pop('Dialogue_Act')
#validlabels=validlabels.values
validfeatures= silicone_data_valid.copy()
validfeatures.drop(columns=['Speaker','Label','Dialogue_Act','Idx'],inplace=True)
validfeatures=validfeatures.values

testfeatures= silicone_data_test.copy()
testfeatures.drop(columns=['Speaker','Label','Dialogue_Act','Idx'],inplace=True)

testfeatures=testfeatures.values


In [49]:
trainfeatures = np.concatenate(trainfeatures, axis = 0)
validfeatures = np.concatenate(validfeatures, axis = 0)
testfeatures = np.concatenate(testfeatures, axis = 0)

In [50]:
from transformers import DistilBertTokenizerFast

In [51]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [52]:

import torch
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

In [53]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [54]:
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer_DistilBERT = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [56]:
dropout_num=0
max_length=160
max_sequence_length=512
learning_rate_BERT=6e-6
val_split_BERT=0.2
epochs_BERT=5
batch_size_BERT=8


In [57]:
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

You can use the raw model for either masked language modeling or next sentence prediction, but it's mostly intended to be fine-tuned on a downstream task. See the model hub to look for fine-tuned versions on a task that interests you.

Note that this model is primarily aimed at being fine-tuned on tasks that use the whole sentence (potentially masked) to make decisions, such as sequence classification, token classification or question answering

In [58]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
    
    return np.array(all_tokens)

In [59]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [60]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(42, activation='sigmoid')(cls_token) # for oasis
    #out = Dense(5, activation='sigmoid')(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss=loss, metrics=['accuracy'])
    
    return model



In [61]:
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [62]:
model = build_model(transformer_layer, max_len=160)

In [63]:
train_input = bert_encode(trainfeatures, tokenizer, max_len=160)
test_input = bert_encode(testfeatures, tokenizer, max_len=160)
train_labels = trainlabels

In [64]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 160)]            0         
                                                                 
 tf_distil_bert_model_3 (TFD  TFBaseModelOutput(last_h  66362880 
 istilBertModel)             idden_state=(None, 160,             
                             768),                               
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 tf.__operators__.getitem_1   (None, 768)              0         
 (SlicingOpLambda)                                               
                                                                 
 dense_1 (Dense)             (None, 42)                32298     
                                                           

In [65]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=1,
    batch_size=32
)

/opt/anaconda3/lib/python3.8/site-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


302/302 [==============================] - 3703s 12s/step - loss: 2.9538 - accuracy: 0.1864 - val_loss: 3.1023 - val_accuracy: 0.1668


In [66]:
train_input.shape

(12076, 160)

In [67]:
train_labels.shape

(12076, 42)

References:
- https://www.kaggle.com/code/alexs2020/bert-1024-and-distilbert-comparison

- https://www.kaggle.com/code/xhlulu/disaster-nlp-distilbert-in-tf/notebook
